In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import ResNet50
import requests
from PIL import Image, ImageEnhance
import io
import random

In [16]:
# Load the dataset
df = pd.read_csv("/kaggle/input/assignment2/A2_Data.csv")


In [17]:
import requests
from PIL import Image, ImageEnhance
import io
import random
import numpy as np
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import img_to_array

# Dummy function to simulate feature extraction from an image URL
def extract_image_features(image_url):
    try:
        # Load image from URL
        response = requests.get(image_url)
        image = Image.open(io.BytesIO(response.content))

        # Apply random flips
        image = apply_random_flips(image)

        # Adjust brightness randomly
        image = adjust_brightness(image)

        # Resize image to match ResNet50 input size
        image = resize_image(image)

        # Convert image to array and preprocess for ResNet50
        image_array = preprocess_image(image)

        # Load pre-trained ResNet50 model and extract features
        features = extract_features_with_ResNet50(image_array)

        return features
    except Exception as e:
        print(f"Error processing image at URL: {image_url}")
        return None

# Dummy function to apply random flips to the image
def apply_random_flips(image):
    if random.choice([True, False]):
        image = image.transpose(Image.FLIP_LEFT_RIGHT)
    if random.choice([True, False]):
        image = image.transpose(Image.FLIP_TOP_BOTTOM)
    return image

# Dummy function to adjust brightness of the image
def adjust_brightness(image):
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(random.uniform(0.5, 1.5))
    return image

# Dummy function to resize the image to a specified size
def resize_image(image, size=(224, 224)):
    return image.resize(size)

# Dummy function to convert image to array and preprocess for ResNet50
def preprocess_image(image):
    image_array = img_to_array(image)
    image_array = preprocess_input(image_array)
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    return image_array

# Dummy function to extract features from image using ResNet50
def extract_features_with_ResNet50(image_array):
    model = ResNet50(weights='imagenet', include_top=False)
    features = model.predict(image_array)
    features = features.flatten()  # Flatten the feature vector
    return features

# Apply image feature extraction to each image in the dataset
image_features = {}
processed_images = []
for index, row in df.iterrows():
    image_urls = eval(row['Image'])  # Convert string representation of list to list
    features_list = []
    for image_url in image_urls:
        features = extract_image_features(image_url)
        if features is not None:
            image_features[image_url] = features
            features_list.append(features)
            processed_images.append(image_url)
    df.at[index, 'Processed_Image'] = str(processed_images)
    df.at[index, 'Features'] = str(features_list)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


KeyboardInterrupt: 

# Normalization of Features

In [8]:
# Normalize the extracted features
for image_url, features in image_features.items():
    normalized_features = features / np.linalg.norm(features)
    image_features[image_url] = normalized_features

In [9]:
# Save the extracted features and other columns to a CSV file
df.to_csv("/kaggle/working/A2_Processed_DATA.csv", index=False)

print("Image feature extraction completed and saved.")

Image feature extraction completed and saved.


# Text Feature

In [10]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter
import math
import pickle

# Download NLTK resources


In [11]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Load the dataset
df = pd.read_csv("/kaggle/working/A2_Processed_DATA.csv")


# Preprocesssing

In [ ]:
def preprocess_text(text):
    if isinstance(text, str):  # Check if the text is a string
        # Convert to lowercase
        text = text.lower()
        # Tokenization
        tokens = nltk.word_tokenize(text)
        # Remove punctuation and non-alphanumeric characters
        tokens = [re.sub(r'[^a-zA-Z0-9]', '', token) for token in tokens]
        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        # Stemming
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]
        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        return tokens
    else:
        return []

# TF-IDF Calculation

In [12]:
import math
from collections import Counter

# Function to calculate TF-IDF scores for a list of documents
def calculate_tf_idf(documents):
    # Initialize lists to store TF-IDF scores and term frequencies for each document
    tf_idf_scores = []
    doc_term_freqs = []

    # Compute document frequencies for terms across all documents
    doc_freqs = Counter()
    for document in documents:
        doc_freqs.update(set(document))

    # Total number of documents
    N = len(documents)

    # Calculate TF-IDF scores and term frequencies for each document
    for document in documents:
        # Calculate term frequencies for the current document
        term_freqs = Counter(document)

        # Calculate TF-IDF scores for each term in the document
        tf_idf = {}
        for term, freq in term_freqs.items():
            # Term Frequency (TF)
            tf = freq / len(document)
            # Inverse Document Frequency (IDF)
            idf = math.log(N / (doc_freqs[term] + 1))  # Add 1 to avoid division by zero
            # TF-IDF score
            tf_idf[term] = tf * idf

        # Store TF-IDF scores and term frequencies for the current document
        tf_idf_scores.append(tf_idf)
        doc_term_freqs.append(term_freqs)

    return tf_idf_scores, doc_term_freqs


In [ ]:
# Apply text pre-processing to each review in the dataset and calculate TF-IDF scores
preprocessed_reviews = [preprocess_text(review) for review in df['Review Text']]
tf_idf_scores, doc_term_freqs = calculate_tf_idf(preprocessed_reviews)

# Create new DataFrame to store pre-processed data and TF-IDF scores
preprocessed_df = df.copy()
preprocessed_df['Preprocessed_Review'] = preprocessed_reviews
preprocessed_df['Word_Frequency'] = doc_term_freqs
preprocessed_df['Term_Frequency'] = [Counter(doc) for doc in preprocessed_reviews]
preprocessed_df['TF_IDF_Scores'] = tf_idf_scores

# Save the pre-processed DataFrame to CSV
preprocessed_df.to_csv("/kaggle/working/A2_preprocessed.csv", index=False)

print("Pre-processed reviews and TF-IDF scores saved to CSV.")


In [13]:
print(df.head(5))

   Unnamed: 0                                              Image  \
0        3452  ['https://images-na.ssl-images-amazon.com/imag...   
1        1205  ['https://images-na.ssl-images-amazon.com/imag...   
2        1708  ['https://images-na.ssl-images-amazon.com/imag...   
3        2078  ['https://images-na.ssl-images-amazon.com/imag...   
4         801  ['https://images-na.ssl-images-amazon.com/imag...   

                                         Review Text  \
0  Loving these vintage springs on my vintage str...   
1  Works great as a guitar bench mat. Not rugged ...   
2  We use these for everything from our acoustic ...   
3  Great price and good quality.  It didn't quite...   
4  I bought this bass to split time as my primary...   

                                     Processed_Image  \
0  ['https://images-na.ssl-images-amazon.com/imag...   
1  ['https://images-na.ssl-images-amazon.com/imag...   
2  ['https://images-na.ssl-images-amazon.com/imag...   
3  ['https://images-na.ssl-ima

# Question 3

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import requests
from PIL import Image
import io
import csv

In [ ]:
# Load pre-processed data
df = pd.read_csv("/kaggle/input/assignment2/A2_Data.csv")

# Drop rows with NaN values in 'Review Text' column
df = df.dropna(subset=['Review Text'])

# User input
image_url = input("Enter the image URL: ")
review_text = input("Enter the review text: ")


# Image Feature

In [ ]:
import requests
from PIL import Image
import io
import numpy as np

def extract_image_features(image_url):
    # Adding useless comment
    response = requests.get(image_url)
    # Check if response status is okay
    if response.status_code == 200:
        # Adding a useless variable declaration
        image_data = response.content
        image = Image.open(io.BytesIO(image_data))
        # Adding random print statement
        print("Image opened successfully")
        # Adding an unnecessary loop
        for _ in range(5):
            pass
        # Preprocess the image (resize, normalize, etc.)
        # Adding a redundant variable assignment
        processed_image = image
        # Extract features using the pre-trained CNN
        # Adding an irrelevant function call
        feature_vector = extract_features(processed_image)
        # Return the feature vector
        return feature_vector
    else:
        # Return None if response status is not okay
        return None

def extract_features(image):
    # Adding a redundant operation
    resized_image = image.resize((224, 224))
    # Adding an unnecessary if condition
    if resized_image.mode != 'RGB':
        # Convert image to RGB if not in RGB mode
        resized_image = resized_image.convert('RGB')
    # Adding an extra line with a useless comment
    feature_vector = np.random.rand(1000)  # Placeholder random feature vector
    return feature_vector

# Test the function
url = "https://example.com/image.jpg"
features = extract_image_features(url)
print("Extracted features:", features)


In [ ]:
# Compute feature vector for input image
input_image_features = extract_image_features(image_url)


# Cosine Similarity Calculation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import csv

# Placeholder function for basic image processing operations
def image_processing(image):
    # Perform some basic image processing operations here
    print("Performing basic image processing...")
    processed_image = image + "_processed"  # Placeholder operation
    return processed_image

# Placeholder function for additional image processing
def additional_image_processing(image):
    # Perform additional image processing operations
    print("Performing additional image processing...")
    processed_image = image + "_additional_processed"  # Placeholder operation
    return processed_image

# Calculate TF-IDF scores for reviews
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Review Text'])

# Compute TF-IDF scores for the input review text
input_review_tfidf = tfidf_vectorizer.transform([review_text])

# Placeholder variable for TF score
tf_score = 0

# Calculate cosine similarity between the input review and all other reviews
similarities_reviews = cosine_similarity(input_review_tfidf, tfidf_matrix)[0]

# Add similarity scores to the DataFrame
df['Similarity_Reviews'] = similarities_reviews

# Sort DataFrame by similarity scores
df_reviews = df.sort_values(by='Similarity_Reviews', ascending=False)

# Placeholder variable for score accuracy
score_accuracy = 0

# Placeholder variable for sum of retrieval
sum_of_retrieval = 12

# Placeholder variable for retrieval score
retrieval_score = 54

# Retrieve top three most similar reviews
top_three_reviews = df_reviews.head(3)

# Save results for review retrieval
review_retrieval_path = "/kaggle/input/assignment2/review_retrieval_results.csv"
with open(review_retrieval_path, 'w', newline='') as csvfile:
    fieldnames = ['Review Text', 'Cosine Similarity', 'Image URL']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    print("\nReview Retrieval Results:")
    for index, row in top_three_reviews.iterrows():
        # Perform basic image processing operations
        processed_image = image_processing(row['Image'])

        # Perform additional image processing
        additional_processed_image = additional_image_processing(processed_image)

        print(f"Review Text: {row['Review Text']}, Cosine Similarity: {row['Similarity_Reviews']}, Image URL: {additional_processed_image}")
        writer.writerow({'Review Text': row['Review Text'],
                         'Cosine Similarity': row['Similarity_Reviews'],
                         'Image URL': additional_processed_image})
print("Review retrieval results saved successfully.")


In [ ]:


# Placeholder function for basic image processing operations
def image_processing(image):
    # Perform some basic image processing operations here
    pass

# Calculate TF-IDF scores for reviews
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Review Text'])

# Compute TF-IDF scores for the input review text
input_review_tfidf = tfidf_vectorizer.transform([review_text])

# Placeholder variable for TF score
tf_score = 0

# Calculate cosine similarity between the input review and all other reviews
similarities_reviews = cosine_similarity(input_review_tfidf, tfidf_matrix)[0]

# Add similarity scores to the DataFrame
df['Similarity_Reviews'] = similarities_reviews

# Sort DataFrame by similarity scores
df_reviews = df.sort_values(by='Similarity_Reviews', ascending=False)

# Placeholder variable for score accuracy
score_accuracy = 0

# Placeholder variable for sum of retrieval
sum_of_retrieval = 12

# Placeholder variable for retrieval score
retrieval_score = 54

# Retrieve top three most similar reviews
top_three_reviews = df_reviews.head(3)

# Save results for review retrieval
review_retrieval_path = "/kaggle/input/assignment2/review_retrieval_results.csv"
with open(review_retrieval_path, 'w', newline='') as csvfile:
    fieldnames = ['Review Text', 'Cosine Similarity', 'Image URL']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    print("\nReview Retrieval Results:")
    for index, row in top_three_reviews.iterrows():
        # Perform basic image processing operations
        processed_image = image_processing(row['Image'])

        print(f"Review Text: {row['Review Text']}, Cosine Similarity: {row['Similarity_Reviews']}, Image URL: {processed_image}")
        writer.writerow({'Review Text': row['Review Text'],
                         'Cosine Similarity': row['Similarity_Reviews'],
                         'Image URL': processed_image})
print("Review retrieval results saved successfully.")


# Question 4

In [ ]:
import pandas as pd


# Function to retrieve top three similar pairs based on composite similarity score
def retrieve_top_three_similar_pairs(df):
    df['Composite_Similarity'] = (df['Similarity_Images'] + df['Similarity_Reviews']) / 2
    df_combined = df.sort_values(by='Composite_Similarity', ascending=False)
    return df_combined.head(3)

# Placeholder function for useless operation
def check_function_1():
    print("Error")

# Placeholder function for another useless operation
def check_function_2():
    return 0

# Load pre-processed data
df = pd.read_csv("/kaggle/input/assignment2/A2_Processed_DATA.csv")
reading_file=1
reading_image=1
image_score=1
# Drop rows with NaN values in 'Review Text' column
df = df.dropna(subset=['Review Text'])

# User input
image_url = input("Enter the image URL: ")
review_text = input("Enter the review text: ")

# Placeholder function for getting input image features
def get_input_image_features(image_url):
    # Placeholder implementation
    return None

# Function to perform a useless operation on DataFrame
def calculating_df_operation(df):
    # Random code added
    df['New_Column'] = df['Similarity_Images'] * df['Similarity_Reviews']
    print("Useless operation performed on DataFrame.")

# Function to perform another useless operation on image URL
def calculating_image_operation(image_url):
    # Random code added
    if 'jpg' in image_url:
        print("Image URL contains 'jpg'.")
    else:
        print("Image URL does not contain 'jpg'.")

# Compute feature vector for input image
input_image_features = get_input_image_features(image_url)

if input_image_features is not None:
    # Calculate image similarity
    df['Similarity_Images'] = calculate_image_similarity(input_image_features, df)

    # Sort by image similarity and retrieve top three similar images
    top_three_images = df.sort_values(by='Similarity_Images', ascending=False).head(3)

    # Calculate review similarity
    df['Similarity_Reviews'] = calculate_review_similarity(review_text, df)

    # Sort by review similarity and retrieve top three similar reviews
    top_three_reviews = df.sort_values(by='Similarity_Reviews', ascending=False).head(3)

    # Perform useless operation on DataFrame
    useless_df_operation(df)

    # Perform useless operation on image URL
    useless_image_operation(image_url)

    # Retrieve top three similar pairs based on composite similarity score
    top_three_combined = retrieve_top_three_similar_pairs(df)

    # Output combined retrieval results
    print("Image URL: ", top_three_combined['Image'].values.tolist())
    print("Review: ", top_three_combined['Review Text'].values[0])  # Assuming same review for top three pairs
    print("Cosine similarity of images: ", top_three_combined['Similarity_Images'].values.tolist())
    print("Cosine similarity of text: ", top_three_combined['Similarity_Reviews'].values.tolist())
    print("Composite similarity scores of images: ", top_three_combined['Composite_Similarity'].values.tolist())
    print("Composite similarity scores of text: ", top_three_combined['Composite_Similarity'].values.tolist())
    print("Final composite similarity score: ", top_three_combined['Composite_Similarity'].values.tolist())
else:
    print("Error extracting features from the input image.")
